#전체 과정을 보여주는 코드 입니다. 
# 제출된 모델을 위한 코드를 따로 제출하였습니다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import collections
import random

#Data preperation

In [ ]:
whole_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

In [ ]:
whole_data.head()

#1. model_feature_extraction(Logistic Regression, RaandomForest, SVM)


##1. Data PreProcess


In [ ]:
whole_data['mail'] = whole_data['mail'].str.replace("Subject", "subject")
whole_data_nosubject = whole_data.copy()

In [ ]:
def subject_extraction(train):
  subject_list = []
  mail_list = []

  for mail in train['mail']:
    lst = mail.split("\r\n")

    find_subject = [i for i in range(len(lst)) if ("subject:" in lst[i]) or ("subject :" in lst[i])]

    last_index = find_subject[-1]

    if len(find_subject) > 1:
      subject = []
      mail = []
      for i in range(find_subject[-1]+1):
        subject.append(lst[i])
      for i in range(find_subject[-1]+2,len(lst)):
        mail.append(lst[i])
      
      subject_str = ""
      mail_str = ""
      for i in subject:
        subject_str += i +" "
      for i in mail:
        mail_str += i + " "

    else:
      mail = []
      for i in range(1,len(lst)):
        mail.append(lst[i])
      
      mail_str = ""
      for i in mail:
        mail_str += i + " "
      subject_str = str(lst[0])

    subject_list.append(subject_str)
    mail_list.append(mail_str)

  return subject_list, mail_list

In [ ]:
subject_list, mail_list = subject_extraction(whole_data)
whole_data['subject'] = pd.DataFrame(subject_list)
whole_data['mail'] = pd.DataFrame(mail_list)

In [ ]:
whole_data.head()

In [ ]:
train_data, val_data = train_test_split(whole_data, test_size=0.2, random_state=42)

In [ ]:
print(len(val_data))
print(len(train_data))

In [ ]:
train_data.head()

##2. FeatureExtraction

In [ ]:
stopwords = ['i', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'herself',
"it", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were',
'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',
'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

In [ ]:
def remove_alphabet(data):
  data = data.split()
  result = []
  for w in data:
    if w not in [i for i in 'qwertyuiopasdfghjklzxcvbnm']:
      result.append(w)
  
  return result

def count_word_without_alphabet(data, top_count=10):
    data = remove_alphabet(data)
    result = []
    for w in data:  
        result.append(w) 
    
    counts = collections.Counter(result)
    return counts.most_common(top_count)

def num_words_in_text(isSpam=1):
  tmp = train_data.loc[train_data['label']==isSpam, 'mail'].str.replace('[^a-zA-Z]', ' ').map(count_word_without_alphabet)

  word_count_list = {}

  for index, item in enumerate(tmp):
      for i in range(len(item)):
          a = item[i][0]
          if (a in word_count_list):
              word_count_list[item[i][0]] += 1
          else:
              word_count_list[item[i][0]] = 1
  
  return word_count_list

def is_word_in(train, word):
    sr = train.str.contains(word).astype(int)
    return sr

In [ ]:
# spam 과 nonspam 내에서 단어의 빈도 수를 구함

spam_word_list = num_words_in_text(1)
non_spam_word_list = num_words_in_text(0)

spam_list = sorted(spam_word_list.items(), key = (lambda x:x[1]), reverse = True)
non_spam_list =  sorted(non_spam_word_list.items(), key = (lambda x:x[1]), reverse = True)

for i in list(spam_list):
  if i[0] not in stopwords and i[1] > 2:
    print(i)

In [ ]:
for i in list(non_spam_list):
  if i[0] not in stopwords and i[1] > 2:
    print(i)

In [ ]:
# spam과 nonspam 동시에 등장하는 단어 중 한 쪽 에서만 많이 등장 한 단어들을 정리

duplicate_words = []

for i in spam_word_list:
    if (i in non_spam_word_list):
        duplicate_words.append((i, spam_word_list[i], non_spam_word_list[i], spam_word_list[i] / non_spam_word_list[i]))

duplicate_words_sorted = sorted(duplicate_words, key = (lambda x:x[3]), reverse = True)
print(np.array(duplicate_words_sorted)[:30])

duplicate_words_sorted = sorted(duplicate_words, key = (lambda x:x[3]), reverse = False)
print(np.array(duplicate_words_sorted)[:30])

In [ ]:
def make_feature(train):

    def word_extraction(word_list, feature_object):
        for w in word_list:
            feature_object[w+'_in'] = is_word_in(train['mail'], w)
        return feature_object
    
    email_len = train['mail'].str.len()
    email_sub_len = train['subject'].str.len()
    email_word_num = train['mail'].str.split().str.len()
    email_sub_word_num = train['subject'].str.split().str.len()

    features = pd.DataFrame()

    features['sub_alphabet_num'] = train['subject'].str.replace('[^a-zA-Z]', '').str.len()
    features['sub_alphabet_pro'] = train['subject'].str.replace('[^a-zA-Z]', '').str.len() / email_sub_len

    features['sub_dec_num'] = train['subject'].str.replace('[^0-9]', '').str.len()
    features['sub_dec_pro'] = train['subject'].str.replace('[^0-9]', '').str.len() / email_sub_len


    features['exc_mark_num'] = train['mail'].str.replace('[^!]', '').str.len()
    features['mail_dec_num'] = train['mail'].str.replace('[^0-9]', '').str.len()
    features['mail_dec_pro'] = train['mail'].str.replace('[^0-9]', '').str.len() / email_len  


    relative_words_nonspam = ['deal', 'gas', 'ls', 'will', 'any', 'there', 'day','contract','thanks','effective', 'month','flow','should', 'into','energy','may','please', 'week', 'suite', 'production', 'know','sale']
    features['relative_nonspam_in'] = train['mail'].str.contains('|'.join(relative_words_nonspam)).astype(int)
    features['relative_nonspam_num'] = train['mail'].str.count('|'.join(relative_words_nonspam))
    features['relative_nonspam_pro'] = train['mail'].str.count('|'.join(relative_words_nonspam)) / email_word_num
    features = word_extraction(relative_words_nonspam,features)


    nonspam_most_words = ['enron', 'hpl','gas', 'deal', 'meter', 'attached', 'daren', 'xls', 'mmbtu','flow']
    features['nonspam_in'] = train['mail'].str.contains('|'.join(nonspam_most_words)).astype(int)
    features['nonspam_num'] = train['mail'].str.count('|'.join(nonspam_most_words))
    features['nonspam_pro'] = train['mail'].str.count('|'.join(nonspam_most_words)) / email_word_num
    features = word_extraction(nonspam_most_words,features) 

    
    spam_most_words = ['http', 'com', 'www', 'new','get','want','online','viagra','hello','save','email','info','re','price','professional','penis', 'adobe', 'windows', 'pills','cheap', 'one', 'belize', 'net', 'image', 'cialis']
    features['spam_in'] = train['mail'].str.contains('|'.join(spam_most_words)).astype(int)
    features['spam_num'] = train['mail'].str.count('|'.join(spam_most_words))
    features['spam_pro'] = train['mail'].str.count('|'.join(spam_most_words)) / email_word_num
    features = word_extraction(spam_most_words,features)

    relative_words_spam = ['www', 'online', 'best', 'money', 'save', 'http', 'hello','image','offer','more','over','us','only','com']
    features['relative_spam_in'] = train['mail'].str.contains('|'.join(relative_words_spam)).astype(int)
    features['relative_spam_num'] = train['mail'].str.count('|'.join(relative_words_spam))
    features['relative_spam_pro'] = train['mail'].str.count('|'.join(relative_words_spam)) / email_word_num
    features = word_extraction(relative_words_spam,features)

    features['mail_len'] = train['mail'].str.len()

    return features

train_X = make_feature(train_data)
withSpam = train_X.copy()
withSpam['label'] = train_data['label']

##(1) LogisticRegression

In [ ]:
max_val = 0
best_index_tmp = np.array([])
feature_corr = withSpam.corr()['label'].sort_values().index
feature_corr = feature_corr.drop(['label'])

indexes = [i for i in range(len(feature_corr))]

train_X = make_feature(train_data)
val_X = make_feature(val_data)

for i in range(1, len(feature_corr)):
  for j in range(10):
    model = LogisticRegression()
    random_index = np.array(random.sample(indexes, i))
    random_list = feature_corr[random_index]

    my_model = model.fit(train_X[np.array(random_list)].fillna(0), train_data['label'])
    score = my_model.score(val_X[random_list].fillna(0), val_data['label'])

    if score > max_val:
      best_index_tmp = random_list

final_selected = best_index_tmp

In [ ]:
final_chosen_model = LogisticRegression()
final_model = final_chosen_model.fit(train_X[final_selected].fillna(0), train_data['label'])

prediction = final_model.predict(val_X[final_selected].fillna(0))
label = val_data['label']
acc = accuracy_score(prediction, label)
prec = precision_score(prediction, label)
recall = recall_score(prediction, label)
f1 = f1_score(prediction, label)

print("F1 score: {}, Accuracy: {}, precision: {}, recall: {}".format(f1,acc,prec,recall))

##(2) RandomForestClassifier

In [ ]:
max_val = 0
best_index_tmp = np.array([])
feature_corr = withSpam.corr()['label'].sort_values().index
feature_corr = feature_corr.drop(['label'])

indexes = [i for i in range(len(feature_corr))]

train_X = make_feature(train_data)
val_X = make_feature(val_data)

for i in range(1, len(feature_corr)):
  for j in range(10):
    model = RandomForestClassifier()
    random_index = np.array(random.sample(indexes, i))
    random_list = feature_corr[random_index]

    my_model = model.fit(train_X[np.array(random_list)].fillna(0), train_data['label'])
    score = my_model.score(val_X[random_list].fillna(0), val_data['label'])

    if score > max_val:
      best_index_tmp = random_list

final_selected = best_index_tmp

In [ ]:
final_chosen_model = RandomForestClassifier()
final_model = final_chosen_model.fit(train_X[final_selected].fillna(0), train_data['label'])

prediction = final_model.predict(val_X[final_selected].fillna(0))
label = val_data['label']
acc = accuracy_score(prediction, label)
prec = precision_score(prediction, label)
recall = recall_score(prediction, label)
f1 = f1_score(prediction, label)

print("F1 score: {}, Accuracy: {}, precision: {}, recall: {}".format(f1,acc,prec,recall))

##(3) SVC

In [ ]:
max_val = 0
best_index_tmp = np.array([])
feature_corr = withSpam.corr()['label'].sort_values().index
feature_corr = feature_corr.drop(['label'])

indexes = [i for i in range(len(feature_corr))]

train_X = make_feature(train_data)
val_X = make_feature(val_data)

for i in range(1, len(feature_corr)):
  for j in range(10):
    model = SVC()
    random_index = np.array(random.sample(indexes, i))
    random_list = feature_corr[random_index]

    my_model = model.fit(train_X[np.array(random_list)].fillna(0), train_data['label'])
    score = my_model.score(val_X[random_list].fillna(0), val_data['label'])

    if score > max_val:
      best_index_tmp = random_list

final_selected = best_index_tmp

In [ ]:
final_chosen_model = SVC()
final_model = final_chosen_model.fit(train_X[final_selected].fillna(0), train_data['label'])

prediction = final_model.predict(val_X[final_selected].fillna(0))
label = val_data['label']
acc = accuracy_score(prediction, label)
prec = precision_score(prediction, label)
recall = recall_score(prediction, label)
f1 = f1_score(prediction, label)

print("F1 score: {}, Accuracy: {}, precision: {}, recall: {}".format(f1,acc,prec,recall))

##(4) GaussianNB

In [ ]:
max_val = 0
best_index_tmp = np.array([])
feature_corr = withSpam.corr()['label'].sort_values().index
feature_corr = feature_corr.drop(['label'])

indexes = [i for i in range(len(feature_corr))]

train_X = make_feature(train_data)
val_X = make_feature(val_data)

for i in range(1, len(feature_corr)):
  for j in range(10):
    model = GaussianNB()
    random_index = np.array(random.sample(indexes, i))
    random_list = feature_corr[random_index]

    my_model = model.fit(train_X[np.array(random_list)].fillna(0), train_data['label'])
    score = my_model.score(val_X[random_list].fillna(0), val_data['label'])

    if score > max_val:
      best_index_tmp = random_list

final_selected = best_index_tmp

In [ ]:
final_chosen_model = GaussianNB()
final_model = final_chosen_model.fit(train_X[final_selected].fillna(0), train_data['label'])

prediction = final_model.predict(val_X[final_selected].fillna(0))
label = val_data['label']
acc = accuracy_score(prediction, label)
prec = precision_score(prediction, label)
recall = recall_score(prediction, label)
f1 = f1_score(prediction, label)

print("F1 score: {}, Accuracy: {}, precision: {}, recall: {}".format(f1,acc,prec,recall))

## randomforest with low features

In [ ]:
selected_by_hand = ['nonspam_in','relative_nonspam_pro','nonspam_pro','sub_dec_pro','nonspam_num','relative_spam_pro','spam_pro','http_in','more_in','sub_alphabet_pro','exc_mark_num','relative_spam_num']

In [ ]:
final_chosen_model = RandomForestClassifier()
final_model = final_chosen_model.fit(train_X[final_selected].fillna(0), train_data['label'])

prediction = final_model.predict(val_X[final_selected].fillna(0))
label = val_data['label']
acc = accuracy_score(prediction, label)
prec = precision_score(prediction, label)
recall = recall_score(prediction, label)
f1 = f1_score(prediction, label)

print("F1 score: {}, Accuracy: {}, precision: {}, recall: {}".format(f1,acc,prec,recall))

In [ ]:
test_data = pd.read_csv("./data/test.csv")

test_data['mail'] = test_data['mail'].str.replace("Subject", "subject")
subject_final, mail_final = subject_extraction(test_data)

test_data['subject'] = pd.DataFrame(subject_final)
test_data['mail'] = pd.DataFrame(mail_final)

prediction = final_model.predict(make_feature(test_data)[final_selected].fillna(0))

In [ ]:
answer = test_data.copy()
answer['label'] = np.array(prediction)
del answer['mail']
del answer['subject']

import os

outname = 'test.csv'

outdir = './data'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

answer.to_csv(fullname, index=False)

#2. RNN / LSTM

##1. Data Preperation

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Embedding
from keras import backend as K

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())

    return _f1score
    
def recall(y_target, y_pred):
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))

    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    count_true_positive_false_negative = K.sum(y_target_yn)

    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    return recall

def precision(y_target, y_pred):
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) 
    y_target_yn = K.round(K.clip(y_target, 0, 1)) 

    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    count_true_positive_false_positive = K.sum(y_pred_yn)

    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    return precision

In [ ]:
rnn_whole_data = pd.read_csv("./data/train.csv")

rnn_train_x, rnn_val_x = train_test_split(rnn_whole_data, test_size=0.2, random_state=42)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(rnn_whole_data['mail'])

X_train_encoded = tokenizer.texts_to_sequences(rnn_train_x['mail'])
X_val_encoded = tokenizer.texts_to_sequences(rnn_val_x['mail'])

max_len = max(len(l) for l in X_train_encoded+X_val_encoded)

X_train_padded = pad_sequences(X_train_encoded, maxlen = max_len)
X_val_padded = pad_sequences(X_val_encoded, maxlen = max_len)

word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1
print(vocab_size)

##(1) RNN

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 64)) 
model.add(SimpleRNN(64))
model.add(Dense(1, activation='tanh'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', f1score, recall])

history = model.fit(X_train_padded, rnn_train_x['label'], epochs=5, batch_size=32, validation_split=0.2, callbacks=[early_stop])

In [ ]:
print(model.evaluate(X_val_padded, rnn_val_x['label'])[1])

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
rnn_test_x = pd.read_csv("./data/test.csv")
X_test_encoded = tokenizer.texts_to_sequences(rnn_test_x['subject'] + rnn_test_x['mail'])
X_test_padded = pad_sequences(X_test_encoded, maxlen = max_len)
pred = model.predict(X_test_padded)

In [ ]:
answer = test_data.copy()
answer['label'] = np.round(pred).astype(np.int64)[:,0]
del answer['mail']
del answer['subject']

import os

outname = 'test.csv'

outdir = './data'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

answer.to_csv(fullname, index=False)

##(2) LSTM

In [ ]:
LSTMmodel = Sequential()
LSTMmodel.add(Embedding(4000, 32))
LSTMmodel.add(LSTM(32, dropout=0.1))
LSTMmodel.add(Dense(1, activation='tanh'))

LSTMmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', f1score, recall])

LSTMhistory = LSTMmodel.fit(X_train_padded, rnn_train_x['label'], epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stop])

In [ ]:
print(LSTMmodel.evaluate(X_val_padded, rnn_val_x['label'])[1])

In [ ]:
epochs = range(1, len(LSTMhistory.history['acc']) + 1)
plt.plot(epochs, LSTMhistory.history['loss'])
plt.plot(epochs, LSTMhistory.history['val_loss'])
plt.title('LSTMmodel loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
X_test_encoded = tokenizer.texts_to_sequences(rnn_test_x['mail'])
X_test_padded = pad_sequences(X_test_encoded, maxlen = max_len)
pred = LSTMmodel.predict(X_test_padded)

answer = rnn_test_x.copy()
answer['label'] = np.round(pred).astype(np.int64)[:,0]
del answer['mail']
del answer['subject']

import os

outname = 'test.csv'

outdir = './result'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

answer.to_csv(fullname, index=False)

#3. Distilbert/Bert

In [ ]:
!pip install transformers

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from transformers import Trainer, TrainingArguments
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import BertTokenizerFast, BertForSequenceClassification, BertTokenizer
import torch
import numpy as np
import pandas as pd

In [ ]:
# 데이터셋 준비
whole_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")


bert_whole_data = whole_data.copy()
bert_train_x, bert_val_x = train_test_split(bert_whole_data, test_size=0.2, random_state=42)
bert_test_x = test_data.copy()

In [ ]:
bert_train_list = []
bert_train_label_list = []
for i in bert_train_x['mail']:
  bert_train_list.append(i)
for i in bert_train_x['label']:
  bert_train_label_list.append(i)


bert_val_list = []
bert_val_label_list = []
for i in bert_val_x['mail']:
  bert_val_list.append(i)
for i in bert_val_x['label']:
  bert_val_label_list.append(i)
  
  
bert_test_list = []
bert_test_label_list = []
for i in bert_test_x['mail']:
  bert_test_list.append(i)
  bert_test_label_list.append(0)

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)

  acc = accuracy_score(labels, preds)
  f1 = f1_score(labels, preds)
  recall = recall_score(labels, preds)
  precision = precision_score(labels, preds)

  return { 'accuracy': acc, 'f1_score': f1, "recall": recall, "precision_score": precision }

##(1) Distilbert

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(bert_train_list, truncation=True, padding=True)
val_encodings = tokenizer(bert_val_list, truncation=True, padding=True)
test_encodings = tokenizer(bert_test_list, truncation=True, padding=True)

train_dataset = Dataset(train_encodings, bert_train_label_list)
val_dataset = Dataset(val_encodings, bert_val_label_list)
test_dataset = Dataset(test_encodings, bert_test_label_list)

In [ ]:
model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to("cuda")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=100,
    evaluation_strategy="steps",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
result = trainer.predict(test_dataset)
preds = []

for i in result[0]:
  if i[0] > i[1]:
    preds.append(0)
  else:
    preds.append(1)

label = bert_test_x.copy()

label['label'] = pd.DataFrame(preds)
del label['mail']

import os

outname = 'test.csv'

outdir = './result'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

label.to_csv(fullname, index=False)

#(2)distilbert with kfold

In [ ]:
model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to("cuda")
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

kf = KFold(n_splits=5, random_state=99, shuffle=True)
X = np.array(bert_train_list+bert_val_list)
y = np.array(bert_train_label_list+bert_val_label_list)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=100,
    evaluation_strategy="steps",
)

In [ ]:
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    x_list = []
    y_list = []
    for i in X_train:
      x_list.append(i)
    for i in y_train:
      y_list.append(i)

    train_encoded = tokenizer(x_list, truncation=True, padding=True)
    train_data = Dataset(train_encoded, y_list)

    x_list = []
    y_list = []
    for i in X_test:
      x_list.append(i)
    for i in y_test:
      y_list.append(i)

    val_encoded = tokenizer(x_list, truncation=True, padding=True)
    val_data = Dataset(val_encoded, y_list)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        compute_metrics=compute_metrics,
    )

    trainer.train()

In [ ]:
test_encodings = tokenizer(bert_test_list, truncation=True, padding=True)
test_dataset = Dataset(test_encodings, bert_test_label_list)

result = trainer.predict(test_dataset)
preds = []

for i in result[0]:
  if i[0] > i[1]:
    preds.append(0)
  else:
    preds.append(1)

label = bert_test_x.copy()

label['label'] = pd.DataFrame(preds)
del label['mail']

import os

outname = 'test.csv'

outdir = './result'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

label.to_csv(fullname, index=False)

##(3) Bert

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", do_lower_case=True)

train_encodings = tokenizer(bert_train_list, truncation=True, padding=True)
val_encodings = tokenizer(bert_val_list, truncation=True, padding=True)
test_encodings = tokenizer(bert_test_list, truncation=True, padding=True)

train_dataset = Dataset(train_encodings, bert_train_label_list)
val_dataset = Dataset(val_encodings, bert_val_label_list)
test_dataset = Dataset(test_encodings, bert_test_label_list)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to("cuda")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=100,
    evaluation_strategy="steps",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
result = trainer.predict(test_dataset)
answers = []
for i in result[0]:
  if i[0] > i[1]:
    answers.append(0)
  else:
    answers.append(1)
answer = bert_test_x.copy()
answer['label'] = pd.DataFrame(answers)
del answer['mail']

import os

outname = 'test.csv'

outdir = './result'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

answer.to_csv(fullname, index=False)

##(4) bert with kfold

In [ ]:
kf = KFold(n_splits=5, random_state=99, shuffle=True)
X = np.array(bert_train_list+bert_val_list)
y = np.array(bert_train_label_list+bert_val_label_list)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to("cuda")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=100,
    evaluation_strategy="steps",
)

In [ ]:
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    x_list = []
    y_list = []
    for i in X_train:
      x_list.append(i)
    for i in y_train:
      y_list.append(i)

    train_encoded = tokenizer(x_list, truncation=True, padding=True)
    train_data = Dataset(train_encoded, y_list)

    x_list = []
    y_list = []
    for i in X_test:
      x_list.append(i)
    for i in y_test:
      y_list.append(i)

    test_encoded = tokenizer(x_list, truncation=True, padding=True)
    test_data = Dataset(test_encoded, y_list)

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
    )

    trainer.train()

In [ ]:
result = trainer.predict(test_dataset)
answers = []
for i in result[0]:
  if i[0] > i[1]:
    answers.append(0)
  else:
    answers.append(1)
answer = bert_test_x.copy()
answer['label'] = pd.DataFrame(answers)
del answer['mail']

import os

outname = 'test.csv'

outdir = './result'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

answer.to_csv(fullname, index=False)